In [6]:
using Revise
using FileIO
using GeometryBasics
using TriangleIntersect
using BenchmarkTools
using GLMakie

In [2]:
mesh = load("C:/College/Research/AnisoEFITJulia/MeshFiles/IIWMiniASCII.stl");

In [99]:
function cubeTriIntersect(verts::Vector{AbstractPoint}, cubeMin::AbstractPoint,cubeMax::AbstractPoint)

end

cubeTriIntersect (generic function with 1 method)

In [42]:
function getVertExtrema(verts::Array{Point3{Float32},1})
    mins = [Inf32,Inf32,Inf32]
    maxs = [-Inf32,-Inf32,-Inf32]
    for v in verts
        for i in 1:3
            mins[i] = min(mins[i],v[i])
            maxs[i] = max(maxs[i],v[i])
        end
    end
    return (mins,maxs)
end
function getVertExtrema!(verts::Array{Point3{Float32},1}, maxs::AbstractVector,mins::AbstractVector)
    mins .= Inf32
    maxs .= -Inf32
    for v in verts
        for i in 1:3
            mins[i] = min(mins[i],v[i])
            maxs[i] = max(maxs[i],v[i])
        end
    end
    return (mins,maxs)
end
function voxelMeshSize(bound::AbstractArray, resolution::Integer)
    longest = findmax(bound)[2]
    ppu = resolution/bound[longest]
    return (round.(Int,bound*ppu),longest,1/ppu)
end
function voxelizeMesh(mesh::GeometryBasics.AbstractMesh; resolution::Integer=100)
    verts = Array{Point3{Float32},1}()
    for t in mesh
        for v in t
            push!(metafree(verts),v)
        end
    end
    #Preallocate vectors 
    mins = [Inf32,Inf32,Inf32]
    maxs = [-Inf32,-Inf32,-Inf32]

    numTris = size(mesh,1)
    tris = [[3*i-2,3*i-1,3*i] for i in 1:numTris]
    normals = [t[j].normals for j in 3, t in mesh]

    #Find the extrema for the mesh
    mins,maxs=getVertExtrema!(verts,maxs,mins)
    #Offset the verts so we're in between the origin and maxs
    maxs-=mins
    for i in size(verts,1)
        verts[i]-=mins
    end
    
    #Find the longest dimensions
    gridSize, longestDim, ds = voxelMeshSize(maxs,resolution)
    origin = [1,1,1]
    grid = zeros(Bool,gridSize[1],gridSize[2],gridSize[3])

    #bounding boxes for all triangles
    triBBs = [getVertExtrema(verts[t]) for t in tris]

    triIntBBs = Vector{Tuple{Vector{Int32},Vector{Int32}}}(undef,numTris)
    for i in 1:numTris
        minBound = clamp.(floor.(Int32,triBBs[i][1]/ds),origin,gridSize)
        maxbound = clamp.(ceil.(Int32,triBBs[i][1]/ds),origin,gridSize)
        triIntBBs[i] = (minBound,maxbound)
    end
    display(triIntBBs)
    
    for b in triIntBBs
        for i in b[1][1]:b[2][1]
            for j in b[1][2]:b[2][2]
                for k in b[1][3]:b[2][3]
                    grid[b[1][1]:b[2][1],b[1][2]:b[2][2],b[1][3]:b[2][3]]=true
                end
            end
        end
    end

    return grid
end

display(GLMakie.volume(voxelizeMesh(mesh,resolution=500)))

Triangle(Float32[4.87178, 0.75, 1.8], Float32[4.87178, 0.0, 1.8], Float32[4.8873715, 0.0, 1.7923648])
GeometryBasics.Point{3, Float32}[[4.87178, 0.75, 1.8], [4.87178, 0.0, 1.8], [4.8873715, 0.0, 1.7923648]]
(Float32[4.87178, 0.0, 1.7923648], Float32[4.8873715, 0.75, 1.8])


GLMakie.Screen(...)

1
2
